# Comparaison de performances sur un cas de recherche de mot unique dans un signal (float32 et complex64)

## Imports

In [1]:
import Random
using Plots
using FFTW
using BenchmarkTools
using CUDA
using CUDA.CUFFT
using CUDA.CUBLAS
using ProfileVega
using StatProfilerHTML

## Paramétrage

In [2]:
Fs = 1e6
f0 = -47e3
nrUW = 32
nrSamples = 2^20
nrDisplay = 2^14
snr = 10.0;

## Génération du signal

In [3]:
rng = Random.MersenneTwister(1973)
Random.seed!(1973)
noise = Random.randn(rng, ComplexF32, nrSamples);

In [4]:
create_ol(df, Fs, nr) = ComplexF32.(cispi.(2*(0:(nr-1))*df/Fs))

create_ol (generic function with 1 method)

In [5]:
@btime ol = create_ol(f0, Fs, nrSamples);

  12.944 ms (2 allocations: 8.00 MiB)


In [6]:
@time ol = create_ol(f0, Fs, nrSamples);

  0.012891 seconds (2 allocations: 8.000 MiB)


In [7]:
mapper_qpsk(data) = ComplexF32.(cispi.(data/2 .+ 1/4))

mapper_qpsk (generic function with 1 method)

In [8]:
dataUW = Random.rand(rng, 0:3, nrUW)
symbolsUW = mapper_qpsk(dataUW);

In [9]:
data = Random.rand(rng, 0:3, nrSamples)
symbols = mapper_qpsk(data);

In [10]:
idxStartUW = Random.rand(rng, 1:(nrSamples - nrUW), 1)[1]
symbols[idxStartUW:(idxStartUW + nrUW-1)] = symbolsUW;

In [11]:
sigma = Float32(10^(-snr/20))
signalBaseband = symbols + sigma*noise;

In [12]:
gr()
scatter(signalBaseband[1:nrDisplay], markersize=5, markeralpha=0.1, markerstrokewidth=0)
xaxis!("I")
yaxis!("Q")
title!("Baseband constellation")
plot!(aspect_ratio=1, figsize=(5,5), legend=false)

## Recherche du mot unique par corrélation large (FFT)

In [13]:
uwL = zeros(ComplexF32, nrSamples)
uwL[1:nrUW] = symbolsUW;

### sur CPU

In [14]:
function find_uw_fft_cpu(s::Vector{ComplexF32}, uw::Vector{ComplexF32})::Tuple{Int64, Float32}
    sF = fft(s)
    uwL = zeros(ComplexF32, length(s))
    uwL[1:length(uw)] = uw
    uwLF = fft(uwL)
    cAbs = abs.(ifft(sF .* conj(uwLF)))
    idxMax = argmax(cAbs)
    cMax = cAbs[idxMax]
    idxMax, cMax
end

find_uw_fft_cpu (generic function with 1 method)

In [15]:
@benchmark idxMax, cMax = find_uw_fft_cpu(signalBaseband, symbolsUW)

BenchmarkTools.Trial: 277 samples with 1 evaluation.
 Range (min … max):  16.616 ms … 25.721 ms  ┊ GC (min … max): 0.00% … 5.51%
 Time  (median):     16.936 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.093 ms ±  1.677 ms  ┊ GC (mean ± σ):  3.13% ± 4.20%

   █▄                                                          
  ▄██▅▂▂▁▁▁▁▁▁▁▁▁▁▁▃▆▅▃▁▂▁▁▁▁▄▆▅▁▁▂▂▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂▁▂▄▆▄▃ ▂
  16.6 ms         Histogram: frequency by time        21.2 ms <

 Memory estimate: 52.01 MiB, allocs estimate: 89.

In [16]:
@time idxMax, cMax = find_uw_fft_cpu(signalBaseband, symbolsUW)

  0.026786 seconds (91 allocations: 52.005 MiB)


(772932, 31.239033f0)

In [17]:
println("index : ", idxMax)
println("max : ", cMax)

index : 772932
max : 31.239033


### sur GPU

In [18]:
#CUDA.@allowscalar(true)
function find_uw_fft_gpu(s::Vector{ComplexF32}, uw::Vector{ComplexF32})::Tuple{Int64, Float32}
    sGpu = CuArray(s)
    #uwGpu = CuArray(uw)
    sF = fft(sGpu)
    uwL = CUDA.zeros(ComplexF32, length(s))
    #uwLHeader = view(uwL, 1:length(uw))
    #uwLHeader = uwGpu
    uwL[1:length(uw)] = uw
    uwLF = fft(uwL)
    cAbs = abs.(ifft(sF .* conj(uwLF)))
    idxMax = argmax(cAbs)
    cMax = cAbs[idxMax]
    idxMax, cMax
end

find_uw_fft_gpu (generic function with 1 method)

In [19]:
@benchmark idxMax, cMax = find_uw_fft_gpu(signalBaseband, symbolsUW)

┌ Warning: Performing scalar indexing on task Task (runnable) @0x00007f5219e2dcd0.
│ Invocation of getindex resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore are only permitted from the REPL for prototyping purposes.
│ If you did intend to index this array, annotate the caller with @allowscalar.
└ @ GPUArraysCore ~/.julia/packages/GPUArraysCore/HaQcr/src/GPUArraysCore.jl:106


BenchmarkTools.Trial: 1792 samples with 1 evaluation.
 Range (min … max):  1.990 ms … 14.108 ms  ┊ GC (min … max): 34.61% … 5.28%
 Time  (median):     2.290 ms              ┊ GC (median):    34.96%
 Time  (mean ± σ):   2.786 ms ±  1.536 ms  ┊ GC (mean ± σ):  28.88% ± 7.43%

  ▄██▅▃▁                                                      
  ██████▇▇▇▇▇▇▅▆▆▇▅▆▅▆▇▇▆▆▇▅▆▆▇▆▅▄▅▆▄▄▄▇▄▄▅▅▅▄▁▅▄▅▄▆▅▄▄▄▆▄▅▄ █
  1.99 ms      Histogram: log(frequency) by time     9.71 ms <

 Memory estimate: 21.00 KiB, allocs estimate: 483.

In [20]:
#@profview idxMax, cMax = find_uw_fft_gpu(signalBaseband, symbolsUW)
#@profilehtml idxMax, cMax = find_uw_fft_gpu(signalBaseband, symbolsUW)

### Remarque :
Performance moins bonne que celle du CPU parallélisé. A investiguer...
Warning concernant l'indexation scalaire à investiguer aussi...

In [21]:
@time idxMax, cMax = find_uw_fft_gpu(signalBaseband, symbolsUW)

  0.004418 seconds (489 allocations: 21.500 KiB, 61.55% gc time)


(772932, 31.239012f0)

## Recherche du mot unique par corrélation courte (naïve)

### sur CPU 1 seul process

In [22]:
function find_uw(s::Vector{ComplexF32}, uw::Vector{ComplexF32})::Tuple{Int64, Float32}
    nr = length(s)
    nrUW = length(uw)
    cMax = 0f0
    idxMax = 0
    for k in eachindex(s) #range(1, nr - nrUW + 1)
        if k <= nr - nrUW + 1
            @inbounds @views cAbs = abs(sum(s[k:k + nrUW - 1] .* conj(uw)))
            if cAbs > cMax
                cMax = cAbs
                idxMax = k
            end
        end
    end
    idxMax, cMax
end

find_uw (generic function with 1 method)

### Remarque :
Méthode ci-dessus abandonnée car très peu performante : 210 ms !
Pb supposée : appel à la macro fonction **sum()**. Essayer avec **reduce()** ?

In [23]:
function find_uw(s::Vector{ComplexF32}, uw::Vector{ComplexF32})::Tuple{Int64, Float32}
    nr = length(s)
    nrUW = length(uw)
    cMax = 0f0
    idxMax = 0
    for k in range(1, nr - nrUW + 1)
        idx = k
        su = 0f0im
        @simd for l in eachindex(uw)
            @fastmath @inbounds su += s[idx] * conj(uw[l])
            idx += 1
        end
        cAbs = abs(su)
        if cAbs > cMax
            cMax = cAbs
            idxMax = k
        end
    end
    idxMax, cMax
end

find_uw (generic function with 1 method)

In [24]:
@benchmark idxMax, cMax = find_uw(signalBaseband, symbolsUW)

BenchmarkTools.Trial: 587 samples with 1 evaluation.
 Range (min … max):  8.024 ms …  8.613 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.515 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.515 ms ± 30.326 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                                   ▄██▄▃      
  ▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▃▄▆█████▇▅▃▃ ▃
  8.02 ms        Histogram: frequency by time        8.58 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [25]:
@time idxMax, cMax = find_uw(signalBaseband, symbolsUW)

  0.008104 seconds (2 allocations: 32 bytes)


(772932, 31.23904f0)

### version parallélisée

In [26]:
Threads.nthreads()

16

In [27]:
function find_uw_parallel(s::Vector{ComplexF32}, uw::Vector{ComplexF32})::Tuple{Int64, Float32}
    nr = length(s)
    nrUW = length(uw)
    nrThreads = Threads.nthreads()
    cMax = zeros(Float32, nrThreads)
    idxMax = zeros(Int64, nrThreads)
    Threads.@threads for k in range(1, nr - nrUW + 1)
        idx = k
        su = 0f0im
        @simd for l in eachindex(uw)
            @fastmath @inbounds su += s[idx] * conj(uw[l])
            idx += 1
        end
        cAbs = abs(su)
        if cAbs > cMax[Threads.threadid()]
            @inbounds cMax[Threads.threadid()] = cAbs
            @inbounds idxMax[Threads.threadid()] = k
        end
    end
    idxMaxP = argmax(cMax)
    idxMaxFinal = idxMax[idxMaxP]
    cMaxFinal = cMax[idxMaxP]
    idxMaxFinal, cMaxFinal
end

find_uw_parallel (generic function with 1 method)

In [28]:
@benchmark idxMax, cMax = find_uw_parallel(signalBaseband, symbolsUW)

BenchmarkTools.Trial: 3829 samples with 1 evaluation.
 Range (min … max):  987.084 μs …   3.551 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       1.236 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.302 ms ± 161.937 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

              ▄▄█▃▁                                              
  ▂▂▂▂▂▂▂▂▃▃▄▅█████▆▅▄▄▄▄▄▄▄▄▃▃▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂ ▃
  987 μs           Histogram: frequency by time         1.93 ms <

 Memory estimate: 8.98 KiB, allocs estimate: 99.

In [29]:
@time idxMax, cMax = find_uw_parallel(signalBaseband, symbolsUW)

  0.001886 seconds (106 allocations: 9.141 KiB)


(772932, 31.23904f0)

## Tests

In [30]:
function find_uw_fft_gpu_test(s::Vector{ComplexF32}, uw::Vector{ComplexF32})::Tuple{Int64, Float32}
    sGpu = CuArray(s)
    #uwGpu = CuArray(uw)
    sF = fft(sGpu)
    uwL = CUDA.zeros(ComplexF32, length(s))
    #uwLHeader = view(uwL, 1:length(uw))
    #uwLHeader = uwGpu
    uwL[1:length(uw)] = uw
    uwLF = fft(uwL)
    cAbs = abs.(ifft(sF .* conj(uwLF)))
    #cAbsCpu = Array(cAbs)
    idxMax = argmax(cAbs)
    cMax = cAbs[idxMax]
    idxMax, cMax
end

find_uw_fft_gpu_test (generic function with 1 method)

In [31]:
@btime idxMax, cMax = find_uw_fft_gpu_test(signalBaseband, symbolsUW)

  1.948 ms (483 allocations: 20.95 KiB)


(772932, 31.239012f0)

In [32]:
CUDA.@profile idxMax, cMax = find_uw_fft_gpu_test(signalBaseband, symbolsUW)

┌ Warning: Calling CUDA.@profile only informs an external profiler to start.
│ The user is responsible for launching Julia under a CUDA profiler.
│ 
│ It is recommended to use Nsight Systems, which supports interactive profiling:
│ $ nsys launch julia
└ @ CUDA.Profile ~/.julia/packages/CUDA/N71Iw/lib/cudadrv/profile.jl:82


(772932, 31.239012f0)